In [119]:
import sys
import math
import numpy as np
sys.path.insert(0, '..')
from net_framework import *
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import random
import json

In [120]:
term_data = pd.read_csv('term.txt', sep="\t", header=None)
term_data.columns = ["#Lnum", "#snum", "#cnum", "Term Abbrev"]
term_data.head()

,#Lnum,#snum,#cnum,Term Abbrev
0,1,1,1,LB
1,1,1,2,LB
2,1,1,3,LE
3,1,1,4,WK
4,1,1,5,LF


In [121]:
cnum_data = pd.read_csv('cnum-vhcm-lab-new.txt', sep="\t")
locations = cnum_data[['#cnum']]
locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
locations['Normalized-b'] = (cnum_data['b*'] - cnum_data['b*'].mean())/(cnum_data['b*'] - cnum_data['b*'].mean()).std() * 1/2
display(locations.head(5))

<ipython-input-121-f811cbef4bec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
<ipython-input-121-f811cbef4bec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
<ipython-input-121-f811cbef4bec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

,#cnum,Normalized-L,Normalized-a,Normalized-b
0,141,1.000000,-0.025675,-0.138822
1,274,0.876301,-0.025504,-0.138822
2,129,0.876301,0.070445,-0.106039
3,230,0.876301,0.070101,-0.089951
4,302,0.876301,0.070617,-0.072041


In [122]:
locations = locations.sort_values('#cnum')
chip_num = list(locations['#cnum'])
lab_norm = [[row[2], row[3], row[4]] for row in locations.itertuples()]
print(chip_num)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [123]:
l1 = term_data[term_data.get('#Lnum').eq(1)]
unique_symbols = list(l1['Term Abbrev'].unique())
l1_grouped = l1.groupby('#cnum')['Term Abbrev'].apply(list)
l1_chip_abbrev_percentage = [[(l1_grouped[i + 1].count(abbrev) / len(l1_grouped[i + 1])) for abbrev in unique_symbols] for i in range(len(l1_grouped))]

In [124]:
l1_result = pd.DataFrame(l1_chip_abbrev_percentage)
l1_result.index += 1
l1_result.index.name = '#cnum'
l1_result.columns = unique_symbols
print(l1_result)
chip_norm = []
#pull the percentage for each cnum
for x in chip_num:
    chip_norm.append((l1_result.loc[l1["#cnum"]==x]).values.tolist()[0])

         LB    LE    WK    LF     F     G     S   GB    FU
#cnum                                                     
1      0.36  0.00  0.00  0.04  0.08  0.52  0.00  0.0  0.00
2      0.12  0.00  0.04  0.12  0.60  0.04  0.00  0.0  0.08
3      0.00  0.96  0.04  0.00  0.00  0.00  0.00  0.0  0.00
4      0.28  0.00  0.40  0.00  0.08  0.04  0.20  0.0  0.00
5      0.04  0.00  0.00  0.20  0.68  0.08  0.00  0.0  0.00
...     ...   ...   ...   ...   ...   ...   ...  ...   ...
326    0.08  0.20  0.36  0.00  0.20  0.00  0.12  0.0  0.04
327    0.04  0.00  0.00  0.68  0.20  0.04  0.04  0.0  0.00
328    0.64  0.00  0.08  0.00  0.00  0.20  0.08  0.0  0.00
329    0.08  0.08  0.40  0.00  0.12  0.00  0.28  0.0  0.04
330    0.04  0.00  0.00  0.72  0.24  0.00  0.00  0.0  0.00

[330 rows x 9 columns]


In [125]:
node_num = range(10, 130, 10)
layer_num = range(1,6)
def mirror(arr, desired_length):
    if arr == [4,4,4]:
        print("this")
    if desired_length/len(arr) == 2:
        return arr + arr[::-1]
    else:
        return arr[:-1] + arr[::-1]

shape_collection = []
for node in node_num:
    for layer in layer_num:
        shape = []
        # it gets stuck on this particular one for some reason
        if node == 20 and layer == 5:
            shape = [4,4,4,4,4]
        if node//layer < 3:
            continue
        elif layer%2 == 0:
            # you are not leaving until you give me the right number
            while sum(shape) != node/2:
                shape = []
                # let the code randomly assign things that add up to what we want
                for l in range (0,int(layer/2)) :
                    shape.append(int(random.uniform(3,node+1)))
            
            shape = mirror(shape, layer)
            shape_collection.append(shape)
        else:
            while sum(shape) != node:
                shape = []
                # similar logic, but we let it mirror itself first
                for l in range (0,(layer//2) + 1) :
                    shape.append(int(random.uniform(3,node+1)))
                    shape = mirror(shape, layer)
                    
            shape_collection.append(shape)

In [126]:
#Number of training iterations
num_iters = 500

#Listing out the shapes of each model
colors_num = len(chip_norm[0])
input_size = 3

network_shapes = []
for s in shape_collection:
    network_shapes.append((input_size,s,colors_num))

#Learning rate of the network
rate = 0.001

#Generating Training Data
input_train, output_train, input_test, output_test = 0, 0, 0, 0
def shuffle():
    lab_train, lab_test, chip_train, chip_test = train_test_split(lab_norm, chip_norm, test_size=0.1, random_state = 3, shuffle = True)
    #test run on whole dataset
    #lab_train, lab_test = lab_norm, lab_norm
    #chip_train, chip_test = chip_norm, chip_norm
    
    global input_train, output_train, input_test, output_test
    input_train = torch.FloatTensor(lab_train)
    output_train = torch.FloatTensor(chip_train)
    input_test= torch.FloatTensor(lab_test)
    output_test = torch.FloatTensor(chip_test)


In [127]:
#Array of losses over training period for each network
for net_num, shape in enumerate(network_shapes):
    shuffle()
    print(shape)
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    

    for i in range(num_iters):
        #Calculating l1 error
        error = NN.l1error(output_train, NN(input_train))
        if i == 0: 
            dh = display("#" + str(i) + " Error: " + str(error), display_id=True)
        else:
            dh.update("#" + str(i) + " Error: " + str(error))
            
        NN.train(input_train, output_train)
    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num));

(3, [10], 9)
Network Shape:
Input Size = 3
Hidden Size = [10]
Output Size = 9


'#499 Error: 0.09922836720943451'

(3, [5, 5], 9)
Network Shape:
Input Size = 3
Hidden Size = [5, 5]
Output Size = 9


'#499 Error: 0.10106920450925827'

(3, [3, 4, 3], 9)
Network Shape:
Input Size = 3
Hidden Size = [3, 4, 3]
Output Size = 9


...

'#499 Error: 0.019175900146365166'

(3, [45, 45], 9)
Network Shape:
Input Size = 3
Hidden Size = [45, 45]
Output Size = 9


'#434 Error: 0.006195578724145889'

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [131]:
output_file = []
for net_num, shape in enumerate(network_shapes):
    try:
        shuffle()
        row = []
        print('Network Shape:',flush = True)
        print('Input Size = ' + str(shape[0]), flush = True)
        print('Hidden Size = ' + str(shape[1]), flush = True)
        print('Output Size = ' + str(shape[2]), flush = True)
        # Loading the network we trained in the prev. section

        NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                            hiddenSize = shape[1] , learning_rate = rate)
        NN.load_state_dict(torch.load("saved_networks/Net " + str(net_num)))

        validation_error = NN.l1error(output_test, NN(input_test))

        max_error = np.max(np.ndarray.flatten(np.abs((output_test - NN(input_test)).detach().numpy())))
        print("The validation error is: " + str(validation_error), flush = True)  
        print("The maximum error is:" + str(max_error), flush = True)
        row.append(shape)
        row.append(validation_error)
        output_file.append(row)
    except Exception:
        pass


    with open('validation_errors.json', 'w') as f:
        json.dump(output_file, f)


Network Shape:
Input Size = 3
Hidden Size = [10]
Output Size = 9
The validation error is: 0.13094303011894226
The maximum error is:0.99999857
Network Shape:
Input Size = 3
Hidden Size = [5, 5]
Output Size = 9
The validation error is: 0.11989116668701172
The maximum error is:0.8715635
Network Shape:
Input Size = 3
Hidden Size = [3, 4, 3]
Output Size = 9
The validation error is: 0.12396591156721115
The maximum error is:1.0
Network Shape:
Input Size = 3
Hidden Size = [20]
Output Size = 9
The validation error is: 0.15149466693401337
The maximum error is:1.0
Network Shape:
Input Size = 3
Hidden Size = [10, 10]
Output Size = 9
The validation error is: 0.1511283814907074
The maximum error is:1.0
Network Shape:
Input Size = 3
Hidden Size = [4, 12, 4]
Output Size = 9
The validation error is: 0.1130281463265419
The maximum error is:0.92116666
Network Shape:
Input Size = 3
Hidden Size = [5, 5, 5, 5]
Output Size = 9
The validation error is: 0.13028186559677124
The maximum error is:0.99998915
Netwo